# Supply Network Design I

## Objective and prerequisites

This notebook focuses on solving a supply network design problem (SNDP) using mathematical optimization. The objective is to find the minimum cost flow through a network that satisfies customer demand while minimizing shipping costs. The problem involves factories, depots, and customers, each with their own capacities and demands.

The notebook provides a step-by-step guide on how to formulate the problem, define the parameters and decision variables, and set up the objective function and constraints. By following this document, you will learn how to apply mathematical optimization techniques to solve real-world supply network design problems.

The model presented here is based on **Example 19** from the fifth edition of *Model Building in Mathematical Programming* by H. Paul Williams (pages 273-275 and 330-332).

## Problem description

In this problem, we have an SNDP with six end customers, each with a known demand for a product. Customer demand can be satisfied from a set of four depots, or directly from a set of two factories. Each node has a maximum capacity: the factories can produce a maximum amount of product, and the depots can support a maximum volume of product moving through them. There are known costs associated with transporting the product along the arcs, from a factory to a depot, from a depot to a customer, or directly from a factory to a customer.

In [ ]:
# Illustration of the network

from IPython.display import Image

Image(filename="figures/sndp-example.png", width=500)


Our supply network has two factories, in Liverpool and Brighton, that produce a product. Each has a maximum production capacity:

| Factory | Supply (tons) |
| --- | --- |
| Liverpool | 150,000 |
| Brighton |  200,000 |

The product can be shipped from a factory to a set of four depots. Each depot has a maximum throughput. Depots don’t produce or consume the product; they simply pass the product on to customers.

| Depot | Throughput (tons) |
| --- | --- |
| Newcastle | 70,000 |
| Birmingham | 50,000 |
| London | 100,000 |
| Exeter | 40,000 |

Our network has six customers, each with a given demand.

| Customer | Demand (tons) |
| --- | --- |
| C1 | 50,000 |
| C2 | 10,000 |
| C3 | 40,000 |
| C4 | 35,000 |
| C5 | 60,000 |
| C6 | 20,000 |

Shipping costs are given in the following table (in dollars per ton): Columns are source nodes and rows are destination nodes, respectively. Thus, for example, it costs $1 per ton to ship the product from Liverpool to London. A `-` in the table indicates that that arc is not possible, so for example it is not possible to ship from the factory in Brighton to the depot in Newcastle.

| To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter |
| --- | --- | --- | --- | --- | --- | --- |
| *Depots* |
| Newcastle  | 0.5 |   - |
| Birmingham | 0.5 | 0.3 |
| London     | 1.0 | 0.5 |
| Exeter     | 0.2 | 0.2 |
| *Customers* |
| C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - |
| C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - |
| C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 |
| C4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 |
| C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 |
| C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 |

The question to be answered is how to satisfy the demands of the end customers while minimizing the total shipping cost.

## Problem formulation

### Sets and indices

- $f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}$

- $d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}\}$

- $c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

- $\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

### Parameters

- $\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

- $\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

- $\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

- $\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).

### Decision variables

$x_{s,t} \in \mathbb{R}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destination $t$.


### Objective function

Minimize the total shipping cost:

$$
\min \ Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{st}\, x_{st}}
$$

### Constraints

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

$$
\sum_{t \in \text{Cities}}{x_{ft}} \leq \text{supply}_{f}, \quad \forall f \in \text{Factories}
$$

- **Customer demand**: Flow of goods must meet customer demand.

$$
\sum_{s \in \text{Cities}}{x_{sc}} = \text{demand}_{c}, \quad \forall c \in \text{Customers}
$$

- **Depot flow**: Flow into a depot equals flow out of the depot.

$$
\sum_{s \in \text{Cities}}{x_{sd}} = \sum_{t \in \text{Cities}}{x_{dt}},\quad \forall d \in \text{Depots}
$$

- **Depot capacity**: Flow into a depot must respect depot capacity.

$$
\sum_{s \in \text{Cities}}{x_{sd}} \leq \text{through}_{d},\quad \forall d \in \text{Depots}
$$



## Pyomo model

In [ ]:
!pip install gurobipy pyomo

import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Include your WSL license information
solver_options = {}

Create objects to store input data:

In [ ]:
supply = {"Liverpool": 150000, "Brighton": 200000}

through = {"Newcastle": 70000, "Birmingham": 50000, "London": 100000, "Exeter": 40000}

demand = {"C1": 50000, "C2": 10000, "C3": 40000, "C4": 35000, "C5": 60000, "C6": 20000}

# Set of nodes
Factories = set(supply.keys())
Depots = set(through.keys())
Customers = set(demand.keys())

print(Factories, Depots, Customers, sep="\n")

In [ ]:
cost = {
    ("Liverpool", "Newcastle"): 0.5,
    ("Liverpool", "Birmingham"): 0.5,
    ("Liverpool", "London"): 1.0,
    ("Liverpool", "Exeter"): 0.2,
    ("Liverpool", "C1"): 1.0,
    ("Liverpool", "C3"): 1.5,
    ("Liverpool", "C4"): 2.0,
    ("Liverpool", "C6"): 1.0,
    ("Brighton", "Birmingham"): 0.3,
    ("Brighton", "London"): 0.5,
    ("Brighton", "Exeter"): 0.2,
    ("Brighton", "C1"): 2.0,
    ("Newcastle", "C2"): 1.5,
    ("Newcastle", "C3"): 0.5,
    ("Newcastle", "C5"): 1.5,
    ("Newcastle", "C6"): 1.0,
    ("Birmingham", "C1"): 1.0,
    ("Birmingham", "C2"): 0.5,
    ("Birmingham", "C3"): 0.5,
    ("Birmingham", "C4"): 1.0,
    ("Birmingham", "C5"): 0.5,
    ("London", "C2"): 1.5,
    ("London", "C3"): 2.0,
    ("London", "C5"): 0.5,
    ("London", "C6"): 1.5,
    ("Exeter", "C3"): 0.2,
    ("Exeter", "C4"): 1.5,
    ("Exeter", "C5"): 0.5,
    ("Exeter", "C6"): 1.5,
}

A = set(cost.keys())  # set of arcs

In [ ]:
# Create the model
mod = pyo.ConcreteModel(name="SNDP")

# Add the decision variables
mod.x = pyo.Var(A, domain=pyo.NonNegativeReals)

# Define the objective function
mod.obj = pyo.Objective(expr=sum(cost[arc] * mod.x[arc] for arc in A), sense=pyo.minimize)

In [ ]:
# Add the supply constraints
mod.supply = pyo.ConstraintList()

for f in Factories:
    expr = sum(mod.x[i, j] for (i, j) in A if i == f)
    mod.supply.add(expr <= supply[f])

# Add the demand constraints
mod.demand = pyo.ConstraintList()

for c in Customers:
    expr = sum(mod.x[i, j] for (i, j) in A if j == c)
    mod.demand.add(expr == demand[c])

# Add the depot capacity constraints
mod.through = pyo.ConstraintList()

for d in Depots:
    expr = sum(mod.x[i, j] for (i, j) in A if i == d)
    mod.through.add(expr <= through[d])

# Add the flow balance constraints
mod.flow = pyo.ConstraintList()

for d in Depots:
    expr1 = sum(mod.x[i, j] for (i, j) in A if i == d)
    expr2 = sum(mod.x[i, j] for (i, j) in A if j == d)
    mod.flow.add(expr1 == expr2)

In [ ]:
# Call the solve and solve the model
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, solver_options=solver_options)
results = opt.solve(mod, tee=True)

## Solution analysis

Product demand from all of our customers can be satisfied for a total cost of

In [ ]:
print(f"${pyo.value(mod.obj):,.2f}")

The optimal plan is as follows:

In [ ]:
import pandas as pd

product_flow = {"From": [], "To": [], "Flow": [], "Cost": []}

for arc in A:
    if pyo.value(mod.x[arc]) > 1e-6:
        product_flow["From"].append(arc[0])
        product_flow["To"].append(arc[1])
        product_flow["Flow"].append(pyo.value(mod.x[arc]))
        product_flow["Cost"].append(cost[arc] * pyo.value(mod.x[arc]))

solution = pd.DataFrame(product_flow)
solution

In [ ]:
# Optional: You can save the solution dataframe to a csv file
solution.to_csv("sndp-sol.csv")

### Exercise

* Which depot experienced the highest throughput?

In [ ]:
# Filter destinations to the depots
solution_d = solution[solution["To"].isin(Depots)]

# calculate the total shipments to each depot
solution_d = solution_d.groupby("To").sum()

solution_d

In [ ]:
# Find the depot with the highest flow

# 1. Order the depots by flow in descending order
solution_d = solution_d.sort_values("Flow", ascending=False)

# 2. Get the name of the depot
depot = solution_d.index[0]

print(f"The depot with the highest flow is: {depot}")



* How much product is produced in each factory? Show the results in a bar chart.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter the origins to the factories
solution_f = solution[solution["From"].isin(Factories)]

# Create a bar chart using seaborn
sns.barplot(data=solution_f, x="From", y="Flow", estimator="sum", errorbar=None)

# Add labels
plt.title("Total Flow Out of Each Factory")
plt.xlabel("Factory")
plt.ylabel("Flow")

# Optional: You can save the plot to an image file
plt.savefig("sndp-factories.png")